In [ ]:
!pip install Flask pyngrok line-bot-sdk requests openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.4/758.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
openai_api_key = userdata.get('OPENAI_API_KEY')
port = 5000

In [ ]:
import os
from pyngrok import ngrok

In [ ]:
ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5000, name="linebot_tunnel")
print(tunnel.public_url)

https://994b-34-86-145-215.ngrok-free.app


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    LocationMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    StickerMessageContent,
    LocationMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text),
                    TextMessage(text=event.message.text)]
            )
        )

@handler.add(MessageEvent, message=StickerMessageContent)
def handle_sticker_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[StickerMessage(
                    package_id=event.message.package_id,
                    sticker_id=event.message.sticker_id)
                ]
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_location_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[LocationMessage(
                    title='定位訊息',
                    address=event.message.address,
                    latitude=event.message.latitude,
                    longitude=event.message.longitude
                )]
            )
        )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/May/2024 08:22:41] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[]}
BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"510116755223085074","quoteToken":"edr-hsveN44twvFvTSwJSLGlfkYhUmvbhilDdDBuMdVB-K6KjM6F1aVof6ugq9bnbbn9xBx0ev6o7XJJkcTpb_bQ6ty2PmBeqN890tH2Uz6oj-VYzwQ-JGhjeLr3dSB8aZj7pmW_0TlpyIS50fNJ5g","text":"hhh "},"webhookEventId":"01HYZ4QDYZ2D7B72DB73RVM3SG","deliveryContext":{"isRedelivery":false},"timestamp":1716884583908,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"99a99aeeaf5946f89fc2b4fa4b864572","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [28/May/2024 08:23:05] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"510116838772834874","quoteToken":"NvPl4bzu8gb0g0cAFafEuZOOYj32XQdGCmAERaqzJA2-6xlLW_9mDuIhbTH9mbeLHGZVihEv-xSWbpos-qf2upjjaqFyepjueyrAfM463YCzswc6efEh9rimwPTq2xYfotDRz-sEPeG1PS_qRRjDGA","text":"ooo"},"webhookEventId":"01HYZ4RYGWWJGQ2AX7QK33KY90","deliveryContext":{"isRedelivery":false},"timestamp":1716884633838,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"00472945fa714328a7e14e35674dc6d5","mode":"active"}]}


...

ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-5-327866b13f63>", line 43, in callback
    handler.handle(body, signature)
  File "/usr/local/lib/python3.10/dist-packages/linebot/v3/webhook.py", line 227, in handle
    self.__invoke_func(func, event, payload)
  File "/usr/local/lib/python3.10/dist-packages/linebot/v3/webhook.py", line 239, in __invoke_func
    func(event)
  File 

BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"sticker","id":"510116976078094678","quoteToken":"3SmoSYyt32VPtFfyb1uzh7gSgi1i4vWudic28E5fPjMkR7CWn_CfjiLPnot1eF5TQ3lwcans_j1_AsbgrvzULxYqzFx919eb31I6l_uezLdEFQjcae5mz5cAR2PsRNgxxogaKwUyYImgzZlnNhNrbg","stickerId":"180","packageId":"3","stickerResourceType":"STATIC","keywords":["Love you","Love","Thanks","smile","Swoon","Happy","cherrycoco","Heart","Beaming","Lovely","Aww","Satisfied"]},"webhookEventId":"01HYZ4VEN4M6AS41Z3C07H0KN5","deliveryContext":{"isRedelivery":false},"timestamp":1716884715689,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"7b390199d0fc4e5d82af696bdaa9527d","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [28/May/2024 08:25:17] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"location","id":"510117056272924937","latitude":25.093839,"longitude":121.546186,"address":"111台灣台北市士林區臨溪路70號東吳大學"},"webhookEventId":"01HYZ4WWWWYXDBZWGMHV6MNRZP","deliveryContext":{"isRedelivery":false},"timestamp":1716884763341,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"b3c3e6796de748389a5e9e4a3c834c83","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [28/May/2024 08:26:04] "POST / HTTP/1.1" 200 -
